In [1]:
import pandas as pd
import numpy as np

In [2]:
uid_train = pd.read_csv('../data/uid_train.txt',sep='\t',header=None,names=('uid','label'))
voice_train = pd.read_csv('../data/voice_train.txt',sep='\t',header=None,names=('uid','opp_num','opp_head','opp_len','start_time','end_time','call_type','in_out'))
sms_train = pd.read_csv('../data/sms_train.txt',sep='\t',header=None,names=('uid','opp_num','opp_head','opp_len','start_time','in_out'))
wa_train = pd.read_csv('../data/wa_train.txt',sep='\t',header=None,names=('uid','wa_name','visit_cnt','visit_dura','up_flow','down_flow','wa_type','date'))

/home/jeff/.conda/envs/py2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2714: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
voice_test = pd.read_csv('../data/voice_test_b.txt',sep='\t',header=None,names=('uid','opp_num','opp_head','opp_len','start_time','end_time','call_type','in_out'))
sms_test = pd.read_csv('../data/sms_test_b.txt',sep='\t',header=None,names=('uid','opp_num','opp_head','opp_len','start_time','in_out'))
wa_test = pd.read_csv('../data/wa_test_b.txt',sep='\t',header=None,names=('uid','wa_name','visit_cnt','visit_dura','up_flow','down_flow','wa_type','date'))

In [4]:
uid_test = pd.DataFrame({'uid':pd.unique(wa_test['uid'])})
uid_test.to_csv('../data/uid_test_b.txt',index=None)

In [5]:
voice = pd.concat([voice_train,voice_test],axis=0)
sms = pd.concat([sms_train,sms_test],axis=0)
wa = pd.concat([wa_train,wa_test],axis=0)

In [6]:
# 对时间进行处理
d1 = np.floor(voice['start_time'] / 1000000)
d2 = np.floor(voice['end_time'] / 1000000)
h1 = np.floor((voice['start_time'] - d1 * 1000000) / 10000)
h2 = np.floor((voice['end_time'] - d2 * 1000000) / 10000)
m1 = np.floor((voice['start_time'] - d1 * 1000000 - h1 * 10000) / 100)
m2 = np.floor((voice['end_time'] - d2 * 1000000 - h2 * 10000) / 100)
s1 = np.floor(voice['start_time'] - d1 * 1000000 - h1 * 10000 - m1 * 100)
s2 = np.floor(voice['end_time'] - d2 * 1000000 - h2 * 10000 - m2 * 100)
# 计算通话时间
voice['time_len'] = (d2 - d1) * 60 * 60 * 60 + (h2 - h1) * 3600 + (m2 - m1) * 60 + s2 -s1
# 计算通话开始时间
voice['start_time_modify'] = h1 + m1 / 60 + s1 / 60 / 60

In [7]:
d = np.floor(sms['start_time'] / 1000000)
h = np.floor((sms['start_time'] - d * 1000000) / 10000)
m = np.floor((sms['start_time'] - d * 1000000 - h * 10000) / 100)
s = np.floor(sms['start_time'] - d * 1000000 - h * 10000 - m * 100)
# 计算发短信开始时间
sms['start_time_modify'] = h + m / 60 + s / 60 / 60

##  baseline 

### 通话记录

In [8]:
voice_opp_num = voice.groupby(['uid'])['opp_num'].agg({'unique_count': lambda x: len(pd.unique(x)),'count':'count'}).add_prefix('voice_opp_num_').reset_index()

voice_opp_head=voice.groupby(['uid'])['opp_head'].agg({'unique_count': lambda x: len(pd.unique(x))}).add_prefix('voice_opp_head_').reset_index()

voice_opp_len=voice.groupby(['uid','opp_len'])['uid'].count().unstack().add_prefix('voice_opp_len_').reset_index().fillna(0)

voice_call_type = voice.groupby(['uid','call_type'])['uid'].count().unstack().add_prefix('voice_call_type_').reset_index().fillna(0)

voice_in_out = voice.groupby(['uid','in_out'])['uid'].count().unstack().add_prefix('voice_in_out_').reset_index().fillna(0)

/home/jeff/.conda/envs/py2/lib/python2.7/site-packages/ipykernel_launcher.py:1: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  """Entry point for launching an IPython kernel.
/home/jeff/.conda/envs/py2/lib/python2.7/site-packages/ipykernel_launcher.py:3: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  This is separate from the ipykernel package so we can avoid doing imports until


In [56]:
# 计算各个用户各个对端号码前n位的通话次数
voice_opp_head_modify = voice.groupby(['uid', 'opp_head'])['uid'].count().unstack().add_prefix('voice_opp_head_').reset_index().fillna(0)

In [43]:
# 通话时长
voice_len = voice.groupby(['uid'])['time_len'].agg(['std','max','min','median','mean','sum']).add_prefix('voice_time_len_').reset_index()
# voice_len = voice.groupby(['uid'])['time_len'].agg(['mean']).add_prefix('voice_time_len_').reset_index()
# 通话时间
voice_time = voice.groupby(['uid'])['start_time_modify'].agg(['std','max','min','median','mean','sum']).add_prefix('voice_start_time_modify_').reset_index()
# voice_time = voice.groupby(['uid'])['start_time_modify'].agg(['std', 'max']).add_prefix('voice_start_time_modify_').reset_index()

## 短信记录

In [11]:
sms_opp_num = sms.groupby(['uid'])['opp_num'].agg({'unique_count': lambda x: len(pd.unique(x)),'count':'count'}).add_prefix('sms_opp_num_').reset_index()

sms_opp_head=sms.groupby(['uid'])['opp_head'].agg({'unique_count': lambda x: len(pd.unique(x))}).add_prefix('sms_opp_head_').reset_index()

sms_opp_len=sms.groupby(['uid','opp_len'])['uid'].count().unstack().add_prefix('sms_opp_len_').reset_index().fillna(0)

sms_in_out = sms.groupby(['uid','in_out'])['uid'].count().unstack().add_prefix('sms_in_out_').reset_index().fillna(0)


/home/jeff/.conda/envs/py2/lib/python2.7/site-packages/ipykernel_launcher.py:1: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  """Entry point for launching an IPython kernel.
/home/jeff/.conda/envs/py2/lib/python2.7/site-packages/ipykernel_launcher.py:3: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  This is separate from the ipykernel package so we can avoid doing imports until


In [15]:
#发信时间
sms_time = sms.groupby(['uid'])['start_time_modify'].agg(['std','max','min','median','mean','sum']).add_prefix('sms_start_time_modify_').reset_index()
# sms_time = sms.groupby(['uid'])['start_time_modify'].agg(['median']).add_prefix('sms_start_time_modify_').reset_index()

### 网站/APP记录

In [13]:
wa_name = wa.groupby(['uid'])['wa_name'].agg({'unique_count': lambda x: len(pd.unique(x)),'count':'count'}).add_prefix('wa_name_').reset_index()
visit_cnt = wa.groupby(['uid'])['visit_cnt'].agg(['std','max','min','median','mean','sum']).add_prefix('wa_visit_cnt_').reset_index()

visit_dura = wa.groupby(['uid'])['visit_dura'].agg(['std','max','min','median','mean','sum']).add_prefix('wa_visit_dura_').reset_index()

up_flow = wa.groupby(['uid'])['up_flow'].agg(['std','max','min','median','mean','sum']).add_prefix('wa_up_flow_').reset_index()

down_flow = wa.groupby(['uid'])['down_flow'].agg(['std','max','min','median','mean','sum']).add_prefix('wa_down_flow_').reset_index()


/home/jeff/.conda/envs/py2/lib/python2.7/site-packages/ipykernel_launcher.py:1: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  """Entry point for launching an IPython kernel.


In [57]:
feature = [voice_opp_num,voice_opp_head,voice_opp_len,voice_call_type,voice_in_out,sms_opp_num,sms_opp_head,sms_opp_len,sms_in_out,wa_name,visit_cnt,visit_dura,up_flow,
           down_flow, voice_opp_head_modify]

# feature = [voice_opp_num,voice_opp_head,voice_opp_len,voice_call_type,voice_in_out,sms_opp_num,sms_opp_head,sms_opp_len,sms_in_out,wa_name,visit_cnt,visit_dura,up_flow,
#            down_flow, voice_len, voice_time, sms_time, voice_opp_head1]

In [58]:
train_feature = uid_train
for feat in feature:
    train_feature=pd.merge(train_feature,feat,how='left',on='uid')

In [59]:
test_feature = uid_test
for feat in feature:
    test_feature=pd.merge(test_feature,feat,how='left',on='uid')

In [60]:
train_feature.to_csv('../data/train_featureV2.csv',index=None)
test_feature.to_csv('../data/test_featureV2.csv',index=None)